# Initialize

In [1]:
!pip install -U spacy

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

In [19]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import transform_text_service_input_to_spacy_format

from transform_text_service_input_to_spacy_format import get_data_from_text_service_item_url

importlib.reload(transform_text_service_input_to_spacy_format)

################
################

get_data_from_text_service_item_url("http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/2")

#get_data_from_text_service_item_url("http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/10/sections/104")

{'api_domain': 'sdir-d-aks-core.norwayeast.cloudapp.azure.com',
 'api_version': 'v1',
 'regularion_variable_name': 'regulations',
 'regularion_year': '2013',
 'regularion_month': '11',
 'regularion_day': '22',
 'regularion_id': '1404',
 'chapter_variable_name': 'chapters',
 'chapter_number': '3',
 'section_variable_name': 'sections',
 'section_number': '15',
 'part_variable_name': 'subsections',
 'part_number': '1',
 'sub_part_variable_name': 'sentences',
 'sub_part_number': '2'}

# Loop through text service chapter input

In [21]:
text_service_chapter_input = {
    "sections": [
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Dette kapittelet gjelder for fartøy bygget etter 2. januar 1988.",
                            "uid": "5305cb7c-b652-499e-4032-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "cd8c26ca-463e-4067-a80c-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Fartøy som er bygget før 2. januar 1988 skal være bygget og utrustet etter reglene som gjaldt da fartøyet ble bygget, med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.",
                            "uid": "19cc03a1-cc8c-4f3b-4033-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "4c451fb1-dbbc-4240-a80d-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Når et fartøy forandres eller ombygges i vesentlig omfang, skal fartøyet oppfylle kravene til nytt fartøy.",
                            "uid": "8db740b9-25ed-44e4-4034-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/3/sentences/1"
                        }
                    ],
                    "uid": "80cc92d1-0509-4f42-a80e-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/3"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/7",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p7",
            "title": "Kapittelets anvendelsesområde",
            "uid": "b141219d-06f4-4326-ccb4-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Fartøy bygget etter 1. januar 1992 skal brannsikres konstruksjonsmessig i henhold til kapittel Y14 i Nordisk Båt Standard (1990), med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.",
                            "uid": "a3445704-dbfc-4ab6-4035-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "5302bfbb-93b1-4fb3-a80f-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Fartøy som er bygget før 1. januar 1992 skal brannsikres konstruksjonsmessig etter reglene som gjaldt da fartøyet ble bygget.",
                            "uid": "0458e04c-4e40-41a7-4036-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "59a29eb2-978c-49e7-a810-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Uavhengig av fartøyets byggedato skal gassfyrte anlegg skal tilfredsstille forskrift 20. oktober 1983 nr. 1580 om sikringstiltak for gassfyrte anlegg m.m. som bruker propan eller andre lette kullvannstoffer anvendt ombord på fartøy.",
                            "uid": "39ca4a94-f8b5-4e48-4037-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8/subsections/3/sentences/1"
                        }
                    ],
                    "uid": "e7779200-07c5-4bca-a811-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8/subsections/3"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/8",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p8",
            "title": "Brannsikring",
            "uid": "3084bd65-4cb9-4919-ccb5-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/8"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Ovner med elektrisk krafttilførsel, skal ha en oljetilførsel som stenges automatisk hvis, ovnen overopphetes (maksimum 400 °C i røykgass), flammen slukker, det oppstår svikt i lufttilførsel eller kraftforsyning.",
                            "uid": "f6d9606e-2969-4f66-4038-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "2a1a151d-82be-4b7c-a812-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Ovner uten elektrisk krafttilførsel, skal ha en oljetilførsel som stenges automatisk hvis, ovnen overopphetes (maksimum 400 °C i røykgass), flammen slukker.",
                            "uid": "d27eaa56-8f14-4723-4039-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "70f5fa4f-6776-42fb-a813-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Ovner med dryppregulering (stillkran med seglass), trenger ikke automatisk stengning av oljetilførsel når flammen slukker ved at oljeforbruket er begrenset av en oljestillerventil.",
                            "uid": "f8848a22-ba48-4fcf-403a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/3/sentences/1"
                        },
                        {
                            "content": "Maksimalt tillatt høydeforskjell mellom oljetank og stillkran skal fremgå av installasjonsbeskrivelsen.",
                            "uid": "87b58366-9f8f-4a55-403b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/3/sentences/2"
                        },
                        {
                            "content": "Oljetrykksreduksjonsventil skal monteres når en tilfredsstillende oljeregulering ikke oppnås på grunn av trykkvariasjoner.",
                            "uid": "66a423aa-7326-49a3-403c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/3/sentences/3"
                        },
                        {
                            "content": "Overflomssystemet skal kunne motta hele tankens volum.",
                            "uid": "44529433-d4f2-4212-403d-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/3/sentences/4"
                        }
                    ],
                    "uid": "27d2ac4a-59cd-48e0-a814-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Tilbakestillingen av sikkerhetsfunksjonen skal være manuell.",
                            "uid": "05657d8b-8a15-4fcf-403e-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/4/sentences/1"
                        }
                    ],
                    "uid": "0f282ffa-7121-44ae-a815-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/4"
                },
                {
                    "sentences": [
                        {
                            "content": "(5) Installasjonsbeskrivelse skal oppbevares om bord.",
                            "uid": "0e948f2b-3bdd-44f0-403f-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/5/sentences/1"
                        },
                        {
                            "content": "Bruksanvisningen skal være oppslått i nærheten av ovnen.",
                            "uid": "8d29323f-3c34-43ad-4040-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/5/sentences/2"
                        }
                    ],
                    "uid": "079f2fbe-b760-4ed8-a816-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/5"
                },
                {
                    "sentences": [
                        {
                            "content": "(6) Ovn med tilbehør skal installeres i overensstemmelse med installasjonsbeskrivelse fra produsent.",
                            "uid": "7bb4102f-193f-4cc3-4041-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/6/sentences/1"
                        }
                    ],
                    "uid": "9be010de-bdb8-4326-a817-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/6"
                },
                {
                    "sentences": [
                        {
                            "content": "(7) Ovn skal ha avtrekk til skorstein.",
                            "uid": "e7e92633-b080-4ec0-4042-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/7/sentences/1"
                        }
                    ],
                    "uid": "4cbefa57-4837-4252-a818-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9/subsections/7"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/9",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p9",
            "title": "Oljefyrt ovn",
            "uid": "a99ffb45-259e-4cf3-ccb6-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/9"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Ovn skal være festet forsvarlig til dørk eller fast benk, om mulig nær fartøyets senterlinje.",
                            "uid": "376cc2ea-ba82-4a6c-4043-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "67adbaa5-b8bf-41d2-a819-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Ovn uten automatisk stengning av oljetilførsel ved slukket flamme, skal ha egen oljetank på maksimalt 40 liter.",
                            "uid": "daf5a7a6-9997-46ee-4044-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/2/sentences/1"
                        },
                        {
                            "content": "Oljetank skal ikke fylles automatisk og skal plasseres i den høyde som er angitt i installasjonsbeskrivelsen.",
                            "uid": "29da6803-35c5-4097-4045-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/2/sentences/2"
                        }
                    ],
                    "uid": "80608ef6-0f0f-4e03-a81a-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Oljetank skal ha vannutskiller og plugg for avtapping av eventuelt vann, og være plassert utenfor det rom hvor ovnen er installert.",
                            "uid": "fcd35cb7-6916-47c7-4046-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/3/sentences/1"
                        }
                    ],
                    "uid": "1a7131e8-28ad-43cb-a81b-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Oljeledningen til regulator skal ha filter for utskilling av slam og vann.",
                            "uid": "297ac8b4-88ef-4c8d-4047-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/4/sentences/1"
                        },
                        {
                            "content": "Filteret skal ikke plasseres over ovnen.",
                            "uid": "3a001816-5e53-4e50-4048-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/4/sentences/2"
                        }
                    ],
                    "uid": "e596fa30-2528-46ee-a81c-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/4"
                },
                {
                    "sentences": [
                        {
                            "content": "(5) Det skal brukes rør av kopper eller likeverdig materiale mellom tank og ovn.",
                            "uid": "190faed9-3d49-44e3-4049-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/5/sentences/1"
                        },
                        {
                            "content": "Rørene skal festes og legges slik at luftlommer unngås.",
                            "uid": "0f3f04c5-de4e-4e57-404a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/5/sentences/2"
                        }
                    ],
                    "uid": "687f76ea-b005-44da-a81d-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/5"
                },
                {
                    "sentences": [
                        {
                            "content": "(6) Ovn skal ha overflomsåpninger med avløpsrør og oppsamlingsbeholder.",
                            "uid": "949306af-8137-4200-404b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/6/sentences/1"
                        },
                        {
                            "content": "Både ovn og eventuell flottørventil skal være tilknyttet oppsamlingsbeholderen.",
                            "uid": "c170b323-5078-4bf2-404c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/6/sentences/2"
                        }
                    ],
                    "uid": "a4732a85-95af-4db0-a81e-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/6"
                },
                {
                    "sentences": [
                        {
                            "content": "(7) Røropptak skal ikke ha mindre diameter enn ovnens avløpsstuss, og røret skal legges med minst mulig bend.",
                            "uid": "d5f15253-26ec-44a7-404d-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/7/sentences/1"
                        }
                    ],
                    "uid": "b079dc7b-04c5-40c2-a81f-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/7"
                },
                {
                    "sentences": [
                        {
                            "content": "(8) Drenering av kondensvann/slam, rensing av filtre, ettersyn av brennere og andre komponenter skal gjøres i tidsintervallene som er beskrevet i bruksanvisningen.",
                            "uid": "ad274d30-2486-4c1e-404e-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/8/sentences/1"
                        }
                    ],
                    "uid": "58960b71-5676-4ccf-a820-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/8"
                },
                {
                    "sentences": [
                        {
                            "content": "(9) I rom med ovn skal det være god ventilasjon.",
                            "uid": "6a196936-ceda-4e3e-404f-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/9/sentences/1"
                        }
                    ],
                    "uid": "ca473934-8e31-4d4b-a821-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/9"
                },
                {
                    "sentences": [
                        {
                            "content": "(10)",
                            "uid": "d088c978-4f39-4d61-4050-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/10/sentences/1"
                        },
                        {
                            "content": "På fartøy med største lengde 8 til 15 meter skal ovnen kontrolleres hos godkjent foretak innen seks måneder etter at den er installert.",
                            "uid": "f54c1597-8981-45c1-4051-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/10/sentences/2"
                        }
                    ],
                    "uid": "bd26c3fe-450b-4598-a822-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/10"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/10",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p10",
            "title": "Installasjon mv. av oljefyrt ovn",
            "uid": "464d118d-3177-4bcd-ccb7-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Skott skal isoleres med ubrennbart materiale i henhold til vedlegg 4.",
                            "uid": "58392aa7-54a9-442e-4052-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/11/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "c077c30f-96a0-4af3-a823-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/11/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Tredørk eller brennbart dekksbelegg under ovn skal beskyttes med stål eller aluminiumsplate under hele ildstedet og 15 cm foran ilegget.",
                            "uid": "5020761b-988d-4e65-4053-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/11/subsections/2/sentences/1"
                        },
                        {
                            "content": "Er ovnen plassert på benk eller liknende, skal benken beskyttes med ubrennbart materiale.",
                            "uid": "a7449a4c-cdbe-43ee-4054-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/11/subsections/2/sentences/2"
                        }
                    ],
                    "uid": "39c77858-7d19-4f41-a824-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/11/subsections/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/11",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p11",
            "title": "Isolasjon av ildsteder mv.",
            "uid": "f0230f49-e2e3-49ba-ccb8-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/11"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Røykopptak og eksosrør skal bestå av minst 3 mm tykke stålplater og festes forsvarlig.",
                            "uid": "46edf0f4-60d2-4a21-4055-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/1/sentences/1"
                        },
                        {
                            "content": "Røykopptaksrør skal føres minst 1,3 m over dekk.",
                            "uid": "8fb08d00-da56-43ec-4056-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/1/sentences/2"
                        }
                    ],
                    "uid": "b4770e8a-7179-4294-a825-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Stenullskåler med minst 6 cm tykkelse skal plasseres mellom røykopptak og brennbart materiale.",
                            "uid": "fbcc25d4-a0bf-4bbc-4057-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/2/sentences/1"
                        },
                        {
                            "content": "Mellom tetningsplater av metall og brennbart materiale skal det isoleres med ubrennbart materiale.",
                            "uid": "4ff53c30-2f5c-4350-4058-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/2/sentences/2"
                        },
                        {
                            "content": "Alternativt kan et rør med minst 6 cm større diameter enn røykopptaksrøret monteres utenpå dette, slik at det blir vertikal luftstrømming mellom rørene.",
                            "uid": "9e4d45a5-235f-4e9a-4059-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/2/sentences/3"
                        },
                        {
                            "content": "Mellom ytre rør og brennbart materiale isoleres med stenullskåler eller mineralullsnor med 3 cm tykkelse.",
                            "uid": "cde800b8-78de-4c40-405a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/2/sentences/4"
                        },
                        {
                            "content": "Røret skal føres minst 1 m over dekk, og utløpet beskyttes med krage.",
                            "uid": "e6b9276f-54c0-48b8-405b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/2/sentences/5"
                        }
                    ],
                    "uid": "f69308a1-b6a6-411a-a826-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Når røykopptak føres opp langs skott, skal skottet isoleres med ubrennbart materiale.",
                            "uid": "2edc28f0-9fc4-4b78-405c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/3/sentences/1"
                        },
                        {
                            "content": "Røykopptak skal ikke plasseres nærmere isolert skott enn 10 cm.",
                            "uid": "fa66f580-8d51-456c-405d-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/3/sentences/2"
                        },
                        {
                            "content": "Avstanden til isolert skott kan reduseres til 5 cm dersom røykopptak (rør) er isolert med stenullskåler eller mineralullsnor av minst 2 cm tykkelse.",
                            "uid": "897e5f57-98fd-4e40-405e-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/3/sentences/3"
                        }
                    ],
                    "uid": "96145d28-7a86-4892-a827-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Eksosmanifold som ikke er vannavkjølt, og deler av eksosrør som kan forårsake antennelse, skal isoleres slik at oljen ikke kan trenge gjennom.",
                            "uid": "13d56501-a76b-4885-405f-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/4/sentences/1"
                        },
                        {
                            "content": "Der gassene er vannavkjølt, kan andre arrangementer godtas.",
                            "uid": "6d989a8b-8df7-4657-4060-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/4/sentences/2"
                        },
                        {
                            "content": "Eksosrør skal ikke være nærmere brennbart materiale enn 2 cm.",
                            "uid": "3cbae0a0-01e2-455e-4061-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/4/sentences/3"
                        }
                    ],
                    "uid": "678bfe30-eb68-4449-a828-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/4"
                },
                {
                    "sentences": [
                        {
                            "content": "(5) Oljeledninger skal legges lengst mulig bort fra eksosrør og varme maskindeler.",
                            "uid": "d07798a5-1b38-473d-4062-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/5/sentences/1"
                        }
                    ],
                    "uid": "5bb9d18c-efee-4da7-a829-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12/subsections/5"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/12",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p12",
            "title": "Røykopptak og eksosrør",
            "uid": "1a9a33e9-7f8d-4ab4-ccb9-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/12"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "Med ubrennbart materiale menes et materiale som ved prøving og vurdering i henhold til MSC.61(67) (FTP-koden) tilfredsstiller kravene til klassifisering av materialer som ubrennbare.",
                            "uid": "6362a584-1682-444b-4063-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/13/subsections/1/sentences/1"
                        },
                        {
                            "content": "Isolasjonen skal om nødvendig, for å bevare den brannisolerende effekten, beslås med tynnplate av galvanisert stål eller aluminium.",
                            "uid": "1331f585-72d9-47c2-4064-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/13/subsections/1/sentences/2"
                        }
                    ],
                    "uid": "3167e43f-9a8c-4da9-a82a-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/13/subsections/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/13",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p13",
            "title": "Isolering med ubrennbart materiale",
            "uid": "01431a74-40d1-4015-ccba-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/13"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Maskinrom i trefartøy skal ha dørk av stål eller aluminiumsplate.",
                            "uid": "ed4aa074-44d4-41ab-4065-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/1/sentences/1"
                        },
                        {
                            "content": "Maskinrom skal isoleres med ubrennbart materiale eller ha et godkjent brannslokningsanlegg.",
                            "uid": "599e0874-7387-4516-4066-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/1/sentences/2"
                        }
                    ],
                    "uid": "836c91a2-7d94-439b-a82b-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Casing av tre, bjelker og underside av dekk og skott i for- og akterkant skal isoleres med ubrennbart materiale i maskinrom uten garnering.",
                            "uid": "4ab9f04b-81d9-4f2d-4067-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "04ec9a9b-ba59-466d-a82c-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Treverk skal isoleres med ubrennbart materiale ned til underkant av dørk i maskinrom med garnering.",
                            "uid": "f45c4260-69d0-40a1-4068-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/3/sentences/1"
                        },
                        {
                            "content": "Dersom den øvrige isolasjon slutter tett inntil og rundt brenselsoljetanken kan isolasjonen bak tanken sløyfes.",
                            "uid": "8417b027-a083-49dc-4069-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/3/sentences/2"
                        }
                    ],
                    "uid": "82de1058-ebc2-45bb-a82d-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Brennbart materiale som utsettes for varme fra eksosmanifolder, eksosrør etc., i fartøy hvor motoren ikke er i et eget rom, skal isoleres med ubrennbart materiale.",
                            "uid": "c9dca5b4-48e4-436f-406a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/4/sentences/1"
                        },
                        {
                            "content": "Brukes det motorkassedeksel, skal denne isoleres innvendig og beslås med galvanisert stål- eller aluminiumsplate.",
                            "uid": "2d9a2894-a66b-4fa8-406b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/4/sentences/2"
                        }
                    ],
                    "uid": "79e48a93-d2f7-4ee5-a82e-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14/subsections/4"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/14",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p14",
            "title": "Isolasjon av maskinrom på trefartøy",
            "uid": "5edbb382-d6ba-4f79-ccbb-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/14"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.",
                            "uid": "115574bc-b56d-4908-406c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/1"
                        },
                        {
                            "content": "Når det er personer om bord skal brannslukkingsanlegget bare kunne utløses manuelt.",
                            "uid": "52b3b082-0808-4240-406d-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/2"
                        }
                    ],
                    "uid": "65afc985-e401-41d6-a82f-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Slokningsmiddel skal være av godkjent type.",
                            "uid": "86fac5c1-5e19-4c08-406e-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/2/sentences/1"
                        },
                        {
                            "content": "Mengden slokningsmiddel fastsettes i hvert enkelt tilfelle avhengig av maskinrommets størrelse.",
                            "uid": "2abcee40-fb65-4af5-406f-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/2/sentences/2"
                        }
                    ],
                    "uid": "3da1d07d-c6b5-4150-a830-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Slokningsanlegget skal bare kunne utløses utenfor maskinrommet.",
                            "uid": "6d4a6d23-0656-4c74-4070-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/3/sentences/1"
                        },
                        {
                            "content": "Utløserutstyret skal være tydelig merket, utstyrt med bruksinstruksjon og beskyttet mot utilsiktet utløsning.",
                            "uid": "9c25046c-a3b1-4df6-4071-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/3/sentences/2"
                        }
                    ],
                    "uid": "4d0be771-425e-403a-a831-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Fartøy med største lengde mindre enn 10,67 meter bygget etter 1. januar 1992 kan alternativt være utstyrt med brannslokningsanlegg i samsvar med retningslinjer fastsatt i kapittel Y14 i Nordisk Båt Standard (1990).",
                            "uid": "d1622523-734c-4460-4072-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/4/sentences/1"
                        }
                    ],
                    "uid": "8c1768ba-5981-48e5-a832-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/4"
                },
                {
                    "sentences": [
                        {
                            "content": "(5) CO2 -beholdere og andre beholdere for slokningsgass som kan medføre fare ved lekkasje, med tilhørende utløsningsanordning, skal være plassert i eget rom/skap utenfor maskinrom.",
                            "uid": "f5370ad3-d091-40cf-4073-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/5/sentences/1"
                        },
                        {
                            "content": "Det skal være gasstette skott til tilstøtende rom og inngang fra åpent dekk.",
                            "uid": "c0d8ecea-0aa2-4288-4074-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/5/sentences/2"
                        },
                        {
                            "content": "Et alarmsignal skal automatisk tre i funksjon når CO2 -rommet/skapet åpnes.",
                            "uid": "c3bd3b3e-8db0-4a2a-4075-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/5/sentences/3"
                        },
                        {
                            "content": "Man må forsikre seg om at ingen personer er tilstede i motorrom før CO2 -gassen utløses.",
                            "uid": "d5269388-a28b-449d-4076-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/5/sentences/4"
                        },
                        {
                            "content": "Rommet/skapet for utløsning av CO2 skal ha ventilasjon til fri luft.",
                            "uid": "ab03beeb-1749-45f4-4077-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/5/sentences/5"
                        }
                    ],
                    "uid": "5b197e13-e72e-42f7-a833-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/5"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/15",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p15",
            "title": "Brannslokningsanlegg i maskinrom",
            "uid": "39a091ee-97e3-4dad-ccbc-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Fartøy mellom største lengde 10,67 og 15 meter, uavhengig av byggeår, skal ha brannoppdagelsesanlegg og brannalarmanlegg for maskinrom og røykvarslingsanlegg for lugarene i samsvar med vedlegg 5.",
                            "uid": "3f171116-0588-4c19-4078-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/16/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "ac2d30a6-a23a-49e8-a834-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/16/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Fartøy mellom største lengde under 10,67 meter skal være utstyrt med minst én røyk- og én varmedetektor i maskinrom, minst én røykdetektor i oppholdsrom, og minst én røykdetektor i styrehus.",
                            "uid": "079e82e9-43dd-4294-4079-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/16/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "c54c29ef-8f01-42c8-a835-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/16/subsections/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/16",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p16",
            "title": "Brannoppdagelses- og alarmanlegg",
            "uid": "4403dadb-7e3c-40dd-ccbd-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/16"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Alle fartøy, uavhengig av byggeår, skal ha minst to godkjente brannslokningsapparat på minst 6 kg.",
                            "uid": "9ba9025b-8f59-4e53-407a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/1/sentences/1"
                        },
                        {
                            "content": "Minst ett godkjent brannslokningsapparat skal plasseres på egnet sted utenfor maskinrommet og minst ett godkjent brannslokningsapparat skal plasseres i innredningen.",
                            "uid": "422d48cf-3089-4947-407b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/1/sentences/2"
                        },
                        {
                            "content": "For fartøy uten styrehus eller innredning er det tilstrekkelig med ett brannslokningsapparat.",
                            "uid": "fc05767e-9ef0-45b5-407c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/1/sentences/3"
                        }
                    ],
                    "uid": "c6684fbb-91ef-4a44-a836-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Brannslukningsapparater med slokningsmiddel under trykk skal ikke plasseres i lugarer.",
                            "uid": "91f8a12b-9c89-4a0f-407d-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/2/sentences/1"
                        },
                        {
                            "content": "Brannslukningsapparater som kan bli utsatt for frost, skal være frostsikre.",
                            "uid": "598eb2f6-cb25-4070-407e-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/2/sentences/2"
                        }
                    ],
                    "uid": "b0f01bba-d284-4620-a837-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Det skal være minst en reserveladning til hvert apparat.",
                            "uid": "65c4c301-d170-43fe-407f-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/3/sentences/1"
                        },
                        {
                            "content": "Apparatenes antall skal økes med 50prosent om det er av en type som ikke kan lades opp ombord.",
                            "uid": "421a4c14-f8a6-41f9-4080-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/3/sentences/2"
                        }
                    ],
                    "uid": "52e19f95-65e7-4689-a838-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Brannslokningsapparater skal årlig kontrolleres i henhold til NS3910.",
                            "uid": "6e74e12a-5bb8-43a0-4081-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/4/sentences/1"
                        },
                        {
                            "content": "Datoen for kontrollen skal påføres apparatet.",
                            "uid": "9bee7eab-1692-4af8-4082-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/4/sentences/2"
                        }
                    ],
                    "uid": "c6274c51-b900-4e49-a839-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17/subsections/4"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/17",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p17",
            "title": "Brannslokningsapparater",
            "uid": "2d22576f-0abc-4f39-ccbe-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "Fartøyets maskinrom skal ha minst to ventilatorer som kan stenges fra dekk.",
                            "uid": "c6de996f-9394-4008-4083-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/18/subsections/1/sentences/1"
                        },
                        {
                            "content": "Lugarer skal ha minst to ventilatorer med diameter på minst 8 cm.",
                            "uid": "361dbeca-6493-4298-4084-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/18/subsections/1/sentences/2"
                        }
                    ],
                    "uid": "a665fbd6-4486-4b7a-a83a-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/18/subsections/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/18",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p18",
            "title": "Ventilasjonssystemer",
            "uid": "e895a34d-9480-4008-ccbf-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/18"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "Materialer, lakk, maling mv. fremstilt på nitrocellulosebasis eller annen lett antennelig basis, tillates ikke benyttet noe sted om bord.",
                            "uid": "6e113739-9d78-4fd0-4085-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/19/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "82509d75-d820-4331-a83b-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/19/subsections/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/19",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p19",
            "title": "Materialer m.m.",
            "uid": "f184195e-1e87-4061-ccc0-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/19"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Brennolje skal oppbevares på tanker av stål eller likeverdig materiale.",
                            "uid": "7e827590-0253-4088-4086-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "f6340654-363b-4f6b-a83c-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Avløp fra brennoljetank skal være utstyrt med avstengningsventil eller kran direkte på tanken, stengbar fra dekk.",
                            "uid": "0fccbacd-95a5-4cd5-4087-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/2/sentences/1"
                        },
                        {
                            "content": "Prøvekraner og kraner for oljestandsglass skal være av selvlukkende type.",
                            "uid": "68bb59a4-5fc3-4d45-4088-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/2/sentences/2"
                        },
                        {
                            "content": "Tanken skal være utstyrt med spillbrett.",
                            "uid": "8a1b8248-5be8-41e0-4089-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/2/sentences/3"
                        },
                        {
                            "content": "På dekket fartøy med maskinrom skal påfyllingsrør og avluftningsrør være ført gjennom dekk ut til fri luft.",
                            "uid": "19d60cdd-50b0-4fdf-408a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/2/sentences/4"
                        }
                    ],
                    "uid": "068ebb8c-5e69-4a29-a83d-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Dersom ekstra brennstoff eller andre brennbare væsker ikke kan anbringes på åpent dekk, skal de oppbevares i egnede beholdere i et godt ventilert rom/skap med dør kun til åpent dekk og uten elektriske eller andre tennkilder.",
                            "uid": "51b47a0f-1f35-4e0e-408b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/3/sentences/1"
                        }
                    ],
                    "uid": "cd63d47d-f8e4-4e3a-a83e-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20/subsections/3"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/20",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p20",
            "title": "Oppbevaring av brennolje og andre brennbare væsker",
            "uid": "4855cf6f-c242-4aa6-ccc1-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/20"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Sjøvannsledninger, lenseledninger, rør som fører olje, og andre rørledninger nødvendige for skipets sikkerhet, skal være av ubrennbart materiale.",
                            "uid": "8105a0d5-8514-4fc1-408c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/21/subsections/1/sentences/1"
                        },
                        {
                            "content": "Korte fleksible rørforbindelser av et materiale som ikke lett gjøres ubrukbart av varme kan godtas på, sjøvannsledninger, korte fleksible rørforbindelser på rør som fører olje når det er nødvendig på grunn av vibrasjoner.",
                            "uid": "f09124d0-632e-4ebf-408d-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/21/subsections/1/sentences/2"
                        }
                    ],
                    "uid": "eb267e39-e754-454d-a83f-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/21/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Spygatt, sanitære avløp eller andre avløp nær vannlinjen hvor svikt i materialet som følge av brann kan forårsake innstrømning av vann skal være av oljebestandige og armerte materialer som ikke lett skades av varme.",
                            "uid": "ec3a0906-c943-4442-408e-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/21/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "d5b7041c-be17-4afa-a840-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/21/subsections/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/21",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p21",
            "title": "Materialer i rørledninger mv.",
            "uid": "ba16334c-075a-4fa3-ccc2-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/21"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Fartøy bygget etter 1. januar 1992 skal oppfylle kravene i Nordisk Båt Standard (1990), Y6 om lensesystemer, Y9 om brennoljesystemer, Y8 om motorinstallasjon, og, Y7 og Y10 om ror og propellarrangement.",
                            "uid": "d65dbcec-ddae-4ae3-408f-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/22/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "255b4c18-3a69-446f-a841-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/22/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Fartøy bygget før 1. januar 1992 skal så langt det er mulig oppfylle kravene i Nordisk Båt Standard (1990) kapittel Y6 om lensesystemer.",
                            "uid": "a39d7beb-9e74-49e6-4090-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/22/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "0602b182-657d-4f5c-a842-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/22/subsections/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/22",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p22",
            "title": "Maskineri og rørsystemer",
            "uid": "4dbe8a87-20e3-4bef-ccc3-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/22"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Fartøy med største lengde 6 til 15 meter bygget etter 1. januar 1992 med elektriske anlegg med spenning inntil 50 V skal oppfylle kravene i Nordisk Båt Standard (1990) Y11 om elektriske anlegg.",
                            "uid": "b6cc0350-45a9-4c82-4091-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "f06dce72-bf43-4b7e-a843-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Fartøy med største lengde 6 til 15 meter bygget før 1. januar 1992 med elektriske anlegg med spenning inntil 50 V skal være i henhold til regelverket som gjaldt ved byggetidspunktet.",
                            "uid": "11f200c0-8c80-434f-4092-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "f23bc866-2adc-4513-a844-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Fartøy med største lengde 6 til 15 meter bygget mellom 1. januar 1992 og 2002 med elektrisk anlegg med spenning over 50 V skal tilfredsstille kravene fastsatt i den nå opphevede forskrift 1. mars 1990 nr. 125 for elektriske anlegg - Maritime installasjoner.",
                            "uid": "b4f5be0a-0747-42f8-4093-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/3/sentences/1"
                        }
                    ],
                    "uid": "42f9a30a-2a7d-4854-a845-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Fartøy med største lengde 6 til 15 meter bygget etter 1. januar 2002 med elektrisk anlegg med spenning over 50 V skal tilfredsstille kravene fastsatt i forskrift 4. desember 2001 nr. 1450 om maritime elektriske anlegg utgitt av Direktoratet for samfunnssikkerhet og beredskap.",
                            "uid": "9110b5cc-c677-4875-4094-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/4/sentences/1"
                        }
                    ],
                    "uid": "f6a1c312-3332-47fa-a846-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23/subsections/4"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/23",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p23",
            "title": "Elektriske anlegg",
            "uid": "70676373-d502-43b5-ccc4-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/23"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "(1) Fartøy med største lengde 10,67 til 15 meter bygget etter 1. januar 1992 skal ha nødkraft som gir tilfredsstillende belysning i minst 2 timer tilstrekkelig til, belysning i område med redningsmidler og utsetningsmateriell, belysning i ganger, trapper og utganger til redningsmidlene og av sjøen hvor redningsmidlene settes ut, nødvendige navigasjonslys, og, pålagt alarmsystem.",
                            "uid": "d2843d54-fa96-4408-4095-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "6826f90d-8ec2-490e-a847-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/1"
                },
                {
                    "sentences": [
                        {
                            "content": "(2) Nødkraften skal komme fra en av følgende to kilder, akkumulatorbatteri som kan tåle belastningen etter første ledd uten oppladning eller for stort spenningsfall og som automatisk kobles til nødstrømtavla når hovedkraften svikter, eller, en generator drevet av en passende kraftmaskin med selvstendig brenselforsyning, en hensiktsmessig startanordning og drevet med brensel med flammepunkt på 43 °C (110 °F) eller mer.",
                            "uid": "0ef8f24e-ef9a-4469-4096-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/2/sentences/1"
                        }
                    ],
                    "uid": "b91d841d-97b0-4511-a848-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/2"
                },
                {
                    "sentences": [
                        {
                            "content": "(3) Hele installasjonen skal fungere selv om skipet har 22,5 grader slagside eller skipets trim er 10 grader.",
                            "uid": "739b45ce-bbea-4013-4097-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/3/sentences/1"
                        }
                    ],
                    "uid": "28e2a442-af6a-40bb-a849-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/3"
                },
                {
                    "sentences": [
                        {
                            "content": "(4) Tavler for hovedkraft og nødkraft skal være tydelig merket og så langt som mulig plasseres slik at de ikke kan utsettes for vann eller ild.",
                            "uid": "7628dfeb-ee35-4a64-4098-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/4/sentences/1"
                        },
                        {
                            "content": "Nødkraftkilden skal om mulig plasseres over dekk.",
                            "uid": "602be633-0777-4091-4099-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/4/sentences/2"
                        },
                        {
                            "content": "Plasseres nødkraftkilden i maskinrommet, skal den plasseres over dørk og på et godt ventilert sted.",
                            "uid": "d657bb20-bdb9-49dc-409a-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/4/sentences/3"
                        }
                    ],
                    "uid": "eb1b724e-54bd-4498-a84a-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/4"
                },
                {
                    "sentences": [
                        {
                            "content": "(5) Nødkraften og nødbelysningen i fartøy bygget før 1. januar 1992 skal være i henhold til regelverket som gjaldt ved byggetidspunktet.",
                            "uid": "b1fd6da8-aa13-47b7-409b-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/5/sentences/1"
                        }
                    ],
                    "uid": "163be831-fe4e-4e89-a84b-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24/subsections/5"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/24",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p24",
            "title": "Nødkraftkilde og nødlys, mv.",
            "uid": "ea69a667-0210-41b4-ccc5-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/24"
        },
        {
            "subsections": [
                {
                    "sentences": [
                        {
                            "content": "Fartøy som er bygget etter Nordisk Båt Standard (1990) anses å tilfredsstille bestemmelsene i § 18, § 19 og § 20.",
                            "uid": "f27163d0-f9c1-4d20-409c-08d8c109f9ce",
                            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/25/subsections/1/sentences/1"
                        }
                    ],
                    "uid": "991f1422-5790-4df1-a84c-08d8c109f9c0",
                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/25/subsections/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/25",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p25",
            "title": "Likeverdige løsninger",
            "uid": "d7a1622b-608d-41d2-ccc6-08d8c109f9b1",
            "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/25"
        }
    ],
    "referenceId": "/kapittel/3",
    "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3",
    "title": "Brann, maskineri og elektro",
    "uid": "da8ef9e9-ce56-49d6-94d3-08d8c109f9af",
    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3"
}

In [34]:
#
# Chapter info
#

print(text_service_chapter_input['referenceId']) # /kapittel/3
print(text_service_chapter_input['referenceUrl']) # https://lovdata.no/forskrift/2013-11-22-1404/k3
print(text_service_chapter_input['title']) # Brann, maskineri og elektro
print(text_service_chapter_input['uid']) # da8ef9e9-ce56-49d6-94d3-08d8c109f9af
print(text_service_chapter_input['url']) # http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/25

number_of_sections_in_chapter = len(text_service_chapter_input['sections'])
print("number_of_sections_in_chapter: " + str(number_of_sections_in_chapter))
print("-------")

for section_index_number, section_item in enumerate(text_service_chapter_input['sections']):
    #section_number = section_index_number + 1
    print(section_item['referenceId'] + " - " + section_item['title'])

    number_of_subsections = len(section_item['subsections'])
    print("number_of_subsections: " + str(number_of_subsections))

    for subsection_index_number, subsection_item in enumerate(section_item['subsections']):
        #"uid": "ca473934-8e31-4d4b-a821-08d8c109f9c0",
        #"url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/9"

        number_of_sentences = len(subsection_item['sentences'])
        print("number_of_sentences: " + str(number_of_sentences))

/kapittel/3
https://lovdata.no/forskrift/2013-11-22-1404/k3
Brann, maskineri og elektro
da8ef9e9-ce56-49d6-94d3-08d8c109f9af
http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3
number_of_sections_in_chapter: 19
-------
/kapittel/3/paragraf/7 - Kapittelets anvendelsesområde
number_of_subsections: 3
number_of_sentences: 1
number_of_sentences: 1
number_of_sentences: 1
/kapittel/3/paragraf/8 - Brannsikring
number_of_subsections: 3
number_of_sentences: 1
number_of_sentences: 1
number_of_sentences: 1
/kapittel/3/paragraf/9 - Oljefyrt ovn
number_of_subsections: 7
number_of_sentences: 1
number_of_sentences: 1
number_of_sentences: 4
number_of_sentences: 1
number_of_sentences: 2
number_of_sentences: 1
number_of_sentences: 1
/kapittel/3/paragraf/10 - Installasjon mv. av oljefyrt ovn
number_of_subsections: 10
number_of_sentences: 1
number_of_sentences: 2
number_of_sentences: 1
number_of_sentences: 2
number_of_sentences: 2
number_of_sentences: 2
number_of_

In [39]:
#
# Explore data
#

print(text_service_chapter_input['referenceId']) # /kapittel/3
print(text_service_chapter_input['referenceUrl']) # https://lovdata.no/forskrift/2013-11-22-1404/k3
print(text_service_chapter_input['title']) # Brann, maskineri og elektro
print(text_service_chapter_input['uid']) # da8ef9e9-ce56-49d6-94d3-08d8c109f9af
print(text_service_chapter_input['url']) # http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/25

number_of_sections_in_chapter = len(text_service_chapter_input['sections'])
print("number_of_sections_in_chapter: " + str(number_of_sections_in_chapter))
print("-------")

for section_index_number, section_item in enumerate(text_service_chapter_input['sections']):
    #section_item['referenceId'] # /kapittel/3/paragraf/17
    #section_item['referenceUrl'] # https://lovdata.no/forskrift/2013-11-22-1404/k3/p17
    #section_item['tables']
    #section_item['title'] # Brannslokningsapparater
    #section_item['uid'] # 2d22576f-0abc-4f39-ccbe-08d8c109f9b1
    #section_item['url'] # http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/17
    
    #print("section_item: " + section_item['uid'])

    #section_number = section_index_number + 1
    #print(section_item['referenceId'] + " - " + section_item['title'])

    #number_of_subsections = len(section_item['subsections'])
    #print("number_of_subsections: " + str(number_of_subsections))

    
    for subsection_index_number, subsection_item in enumerate(section_item['subsections']):
        #subsection_item['sentences']
        #subsection_item['uid']
        #subsection_item['url'] # http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/18/subsections/1

        #number_of_sentences = len(subsection_item['sentences'])
        #print("number_of_sentences: " + str(number_of_sentences))

        #print("subsection_item: " + subsection_item['uid'])

        for sentence_index_number, sentence_item in enumerate(subsection_item['sentences']):
            #sentence_item['content']
            #sentence_item['uid'] # f27163d0-f9c1-4d20-409c-08d8c109f9ce
            #sentence_item['url']

            print("sentence_item: " + sentence_item['uid'])

/kapittel/3
https://lovdata.no/forskrift/2013-11-22-1404/k3
Brann, maskineri og elektro
da8ef9e9-ce56-49d6-94d3-08d8c109f9af
http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3
number_of_sections_in_chapter: 19
-------
sentence_item: 5305cb7c-b652-499e-4032-08d8c109f9ce
sentence_item: 19cc03a1-cc8c-4f3b-4033-08d8c109f9ce
sentence_item: 8db740b9-25ed-44e4-4034-08d8c109f9ce
sentence_item: a3445704-dbfc-4ab6-4035-08d8c109f9ce
sentence_item: 0458e04c-4e40-41a7-4036-08d8c109f9ce
sentence_item: 39ca4a94-f8b5-4e48-4037-08d8c109f9ce
sentence_item: f6d9606e-2969-4f66-4038-08d8c109f9ce
sentence_item: d27eaa56-8f14-4723-4039-08d8c109f9ce
sentence_item: f8848a22-ba48-4fcf-403a-08d8c109f9ce
sentence_item: 87b58366-9f8f-4a55-403b-08d8c109f9ce
sentence_item: 66a423aa-7326-49a3-403c-08d8c109f9ce
sentence_item: 44529433-d4f2-4212-403d-08d8c109f9ce
sentence_item: 05657d8b-8a15-4fcf-403e-08d8c109f9ce
sentence_item: 0e948f2b-3bdd-44f0-403f-08d8c109f9ce
sentence_i

# Functions run from .py file

In [45]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import transform_text_service_input_to_spacy_format

from transform_text_service_input_to_spacy_format import get_data_from_text_service_item_url

importlib.reload(transform_text_service_input_to_spacy_format)

################
################

for section_item in text_service_chapter_input['sections']:    
    for subsection_item in section_item['subsections']:
        for sentence_item in subsection_item['sentences']:
            data = get_data_from_text_service_item_url(sentence_item['url'])
            section_number = str(data['section_number'])
            part_number = str(data['part_number'])
            sub_part_number = str(data['sub_part_number'])

            print(sentence_item['uid'] + " - section: " + section_number + " part: " + part_number + " sub: " + sub_part_number)

5305cb7c-b652-499e-4032-08d8c109f9ce - section: 7 part: 1 sub: 1
19cc03a1-cc8c-4f3b-4033-08d8c109f9ce - section: 7 part: 2 sub: 1
8db740b9-25ed-44e4-4034-08d8c109f9ce - section: 7 part: 3 sub: 1
a3445704-dbfc-4ab6-4035-08d8c109f9ce - section: 8 part: 1 sub: 1
0458e04c-4e40-41a7-4036-08d8c109f9ce - section: 8 part: 2 sub: 1
39ca4a94-f8b5-4e48-4037-08d8c109f9ce - section: 8 part: 3 sub: 1
f6d9606e-2969-4f66-4038-08d8c109f9ce - section: 9 part: 1 sub: 1
d27eaa56-8f14-4723-4039-08d8c109f9ce - section: 9 part: 2 sub: 1
f8848a22-ba48-4fcf-403a-08d8c109f9ce - section: 9 part: 3 sub: 1
87b58366-9f8f-4a55-403b-08d8c109f9ce - section: 9 part: 3 sub: 2
66a423aa-7326-49a3-403c-08d8c109f9ce - section: 9 part: 3 sub: 3
44529433-d4f2-4212-403d-08d8c109f9ce - section: 9 part: 3 sub: 4
05657d8b-8a15-4fcf-403e-08d8c109f9ce - section: 9 part: 4 sub: 1
0e948f2b-3bdd-44f0-403f-08d8c109f9ce - section: 9 part: 5 sub: 1
8d29323f-3c34-43ad-4040-08d8c109f9ce - section: 9 part: 5 sub: 2
7bb4102f-193f-4cc3-4041-0

In [48]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import transform_text_service_input_to_spacy_format

from transform_text_service_input_to_spacy_format import transform_chapter_from_text_service_to_spacy_format

importlib.reload(transform_text_service_input_to_spacy_format)

################
################

transform_chapter_from_text_service_to_spacy_format(text_service_chapter_input)

[{'text': '(1) Dette kapittelet gjelder for fartøy bygget etter 2. januar 1988.',
  'ents': [],
  'title': 'http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/1/sentences/1'},
 {'text': '(2) Fartøy som er bygget før 2. januar 1988 skal være bygget og utrustet etter reglene som gjaldt da fartøyet ble bygget, med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.',
  'ents': [],
  'title': 'http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/2/sentences/1'},
 {'text': '(3) Når et fartøy forandres eller ombygges i vesentlig omfang, skal fartøyet oppfylle kravene til nytt fartøy.',
  'ents': [],
  'title': 'http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/7/subsections/3/sentences/1'},
 {'text': '(1) Fartøy bygget etter 1. januar 1992 skal brannsikres konstruksjonsmessig i henhold til ka